In [ ]:
import pandas as pd
import numpy as np
import re

In [198]:
bundesliga_21_22_df = pd.read_csv(r"all_csv_data\D1 (16).csv")
bundesliga_21_22_df.head()
season = "06"

In [199]:
# calculates home many games a has already played until the matchday in a season
# can be differed between home and away games and between the teams

games_played_this_season_ht_home = []
games_played_this_season_ht_away = []
games_played_this_season_at_home = []
games_played_this_season_at_away = []

for index, rows in bundesliga_21_22_df.iterrows():
    home_team_temp = rows.HomeTeam
    away_team_temp = rows.AwayTeam
    games_played_all_teams_home = bundesliga_21_22_df.iloc[:index].HomeTeam.value_counts()
    games_played_all_teams_away = bundesliga_21_22_df.iloc[:index].AwayTeam.value_counts()
    games_played_this_season_ht_home.append(0 if home_team_temp not in games_played_all_teams_home else games_played_all_teams_home[home_team_temp])
    games_played_this_season_ht_away.append(0 if home_team_temp not in games_played_all_teams_away else games_played_all_teams_away[home_team_temp])
    games_played_this_season_at_home.append(0 if away_team_temp not in games_played_all_teams_home else games_played_all_teams_home[away_team_temp])
    games_played_this_season_at_away.append(0 if away_team_temp not in games_played_all_teams_away else games_played_all_teams_away[away_team_temp])

bundesliga_21_22_df.loc[:,"GPHTH"] = games_played_this_season_ht_home
bundesliga_21_22_df.loc[:,"GPHTA"] = games_played_this_season_ht_away
bundesliga_21_22_df.loc[:,"GPATH"] = games_played_this_season_at_home
bundesliga_21_22_df.loc[:,"GPATA"] = games_played_this_season_at_away

# calculate total games played
bundesliga_21_22_df.loc[:,"GPHT"] = bundesliga_21_22_df.loc[:,"GPHTH"] + bundesliga_21_22_df.loc[:,"GPHTA"]
bundesliga_21_22_df.loc[:,"GPAT"] = bundesliga_21_22_df.loc[:,"GPATH"] + bundesliga_21_22_df.loc[:,"GPATA"]
bundesliga_21_22_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,GPHTH,GPHTA,GPATH,GPATA,GPHT,GPAT
0,D1,11/08/06,Bayern Munich,Dortmund,2,0,H,1,0,H,...,2.17,2.07,1.82,1.77,0,0,0,0,0,0
1,D1,12/08/06,Hamburg,Bielefeld,1,1,D,0,1,A,...,2.02,1.99,1.95,1.88,0,0,0,0,0,0
2,D1,12/08/06,Leverkusen,Aachen,3,0,H,2,0,H,...,1.80,1.77,2.17,2.08,0,0,0,0,0,0
3,D1,12/08/06,Mainz,Bochum,2,1,H,1,0,H,...,1.85,1.81,2.10,2.06,0,0,0,0,0,0
4,D1,12/08/06,M'gladbach,Cottbus,2,0,H,0,0,D,...,2.16,2.09,1.83,1.77,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,D1,19/05/07,Leverkusen,Dortmund,2,1,H,1,0,H,...,2.19,2.10,1.83,1.78,16,17,17,16,33,33
302,D1,19/05/07,M'gladbach,Bochum,0,2,A,0,1,A,...,1.96,1.89,2.02,1.91,16,17,17,16,33,33
303,D1,19/05/07,Schalke 04,Bielefeld,2,1,H,2,0,H,...,2.03,1.99,1.93,1.89,16,17,17,16,33,33
304,D1,19/05/07,Stuttgart,Cottbus,2,1,H,1,1,D,...,1.84,1.80,2.12,2.08,16,17,17,16,33,33


In [200]:
# calculate how many points a team has gained out of the last 3, 5 and 10 matches:
list_number_of_matches = [3, 5, 10, 34]
last_match_points = []

for index, rows in bundesliga_21_22_df.iterrows():
    last_match_points_temp = []
    for i in list_number_of_matches:
        home_team_temp = rows.HomeTeam
        away_team_temp = rows.AwayTeam
        teams_points_temp = []
        for team in [home_team_temp, away_team_temp]:
            if rows.GPHT >= i or i == 34:
                df_last_games_ht = bundesliga_21_22_df[(bundesliga_21_22_df.GPHT < rows.GPHT)]
                if i != 34:
                    df_last_games_ht = df_last_games_ht[(df_last_games_ht.GPHT >= rows.GPHT - i)]
                # calculate all points at home
                df_last_games_specific_ht = df_last_games_ht[(df_last_games_ht.HomeTeam==team)].FTR.value_counts()
                points = 0
                if 'H' in df_last_games_specific_ht:
                    points += 3 * df_last_games_specific_ht['H']
                if 'D' in df_last_games_specific_ht:
                    points += df_last_games_specific_ht['D']
                # calculate all points away
                df_last_games_specific_ht = df_last_games_ht[(df_last_games_ht.AwayTeam==team)].FTR.value_counts()
                if 'A' in df_last_games_specific_ht:
                    points += 3 * df_last_games_specific_ht['A']
                if 'D' in df_last_games_specific_ht:
                    points += df_last_games_specific_ht['D']
                teams_points_temp.append(points)
            else:
                teams_points_temp = [None, None]
        last_match_points_temp.extend(teams_points_temp)
    last_match_points.append(last_match_points_temp)
last_match_points

[[None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 0, 0],
 [None, None, None, None, None, None, 3, 3],
 [None, None, None, None, None, None, 0, 1],
 [None, None, None, None, None, None, 0, 1],
 [None, None, None, None, None, None, 0, 3],
 [None, None, None, None, None, None, 1, 1],
 [None, None, None, None, None, None, 1, 0],
 [None, None, None, None, None, None, 3, 3],
 [None, None, None, None, None, None, 1, 0],
 [None, None, None, None, None, None, 0, 3],
 [None, None, None, None, None, None, 4, 6],
 [None, None, None, None, None, None, 6, 6],
 [None, None, None, None, None, None, 0, 1],
 [None, None, None, None, None, None, 0, 0],
 [None, No

In [201]:
df_bundesliga_with_form = pd.concat([bundesliga_21_22_df, pd.DataFrame(last_match_points, columns=["PointsLast3MatchesHT", "PointsLast3MatchesAT", "PointsLast5MatchesHT", "PointsLast5MatchesAT", "PointsLast10MatchesHT", "PointsLast10MatchesAT","PointsLastAllMatchesHT", "PointsLastAllMatchesAT"])], axis=1)

In [202]:
for i in [3, 5, 10, "All"]:
    df_bundesliga_with_form[f"PDiff{i}Matches"] = df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesHT"] - df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesAT"]
    df_bundesliga_with_form[f"PQuot{i}Matches"] = df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesHT"]/df_bundesliga_with_form.loc[:, f"PointsLast{i}MatchesAT"]

In [203]:
df_bundesliga_with_form

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,PointsLastAllMatchesHT,PointsLastAllMatchesAT,PDiff3Matches,PQuot3Matches,PDiff5Matches,PQuot5Matches,PDiff10Matches,PQuot10Matches,PDiffAllMatches,PQuotAllMatches
0,D1,11/08/06,Bayern Munich,Dortmund,2,0,H,1,0,H,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,D1,12/08/06,Hamburg,Bielefeld,1,1,D,0,1,A,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,D1,12/08/06,Leverkusen,Aachen,3,0,H,2,0,H,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,D1,12/08/06,Mainz,Bochum,2,1,H,1,0,H,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,D1,12/08/06,M'gladbach,Cottbus,2,0,H,0,0,D,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,D1,19/05/07,Leverkusen,Dortmund,2,1,H,1,0,H,...,48,44,-6.0,0.333333,-6.0,0.500000,0.0,1.000000,4,1.090909
302,D1,19/05/07,M'gladbach,Bochum,0,2,A,0,1,A,...,26,42,-5.0,0.166667,-8.0,0.111111,-13.0,0.277778,-16,0.619048
303,D1,19/05/07,Schalke 04,Bielefeld,2,1,H,2,0,H,...,65,42,-6.0,0.333333,-3.0,0.750000,-3.0,0.842105,23,1.547619
304,D1,19/05/07,Stuttgart,Cottbus,2,1,H,1,1,D,...,67,41,6.0,3.000000,9.0,2.500000,6.0,1.375000,26,1.634146


In [204]:
bundesliga_21_22_df.HomeTeam.unique()

array(['Bayern Munich', 'Hamburg', 'Leverkusen', 'Mainz', "M'gladbach",
       'Schalke 04', 'Stuttgart', 'Hannover', 'Wolfsburg', 'Nurnberg',
       'Aachen', 'Cottbus', 'Dortmund', 'Ein Frankfurt', 'Hertha',
       'Werder Bremen', 'Bielefeld', 'Bochum'], dtype=object)

In [205]:
market_values_bl

,club,Squad,ø age,Foreigners,ø market value,Total market value
0,Bayern Munich,30,26. Mai,16,€6.02m,180.73
1,Werder Bremen,33,24. Sep,14,€4.77m,157.25
2,Schalke 04,30,24. Jul,14,€3.55m,106.60
3,Hamburg,36,24. Sep,19,€2.83m,101.83
4,Stuttgart,32,25. Mrz,11,€3.03m,97.08
5,Dortmund,32,24.0,14,€2.49m,79.78
6,Leverkusen,28,25. Jan,14,€2.65m,74.08
7,Hertha,31,25. Apr,18,€2.14m,66.40
8,Nurnberg,31,25. Mai,20,€2.11m,65.45
9,Wolfsburg,30,26. Sep,21,€1.86m,55.90


In [206]:
# add market values
# preprocess csv data with market values
market_values_bl = pd.read_csv(f"market_values_df\\market_values_bl_{season}.csv", delimiter=";")
map_team_names_dict = {'1. FC Köln': 'FC Koln',
 '1.FC Union Berlin': 'Union Berlin',
 '1.FSV Mainz 05': 'Mainz',
 'Arminia Bielefeld': 'Bielefeld',
 'Bayer 04 Leverkusen': 'Leverkusen',
 'Bayern Munich': 'Bayern Munich',
 'Borussia Dortmund': 'Dortmund',
 'Borussia Mönchengladbach': "M'gladbach",
 'Eintracht Frankfurt': 'Ein Frankfurt',
 'FC Augsburg': 'Augsburg',
 'Hertha BSC': 'Hertha',
 'RB Leipzig': 'RB Leipzig',
 'SC Freiburg': 'Freiburg',
 'SpVgg Greuther Fürth': 'Greuther Furth',
 'TSG 1899 Hoffenheim': 'Hoffenheim',
 'VfB Stuttgart': 'Stuttgart',
 'VfL Bochum': 'Bochum',
 'VfL Wolfsburg': 'Wolfsburg',
 'SV Werder Bremen': "Werder Bremen", 
 'FC Schalke 04': "Schalke 04",
 'SC Paderborn 07': 'Paderborn',
 'Fortuna Düsseldorf': 'Fortuna Dusseldorf',
'1.FC Nuremberg': 'Nurnberg',
'Hannover 96': 'Hannover',
'Hamburger SV': 'Hamburg',
'SV Darmstadt 98': 'Darmstadt',
'FC Ingolstadt 04': 'Ingolstadt',
'Eintracht Braunschweig': 'Braunschweig',
'1.FC Kaiserslautern': 'Kaiserslautern',
'FC St. Pauli': 'St Pauli',
'FC Energie Cottbus': 'Cottbus',
'Karlsruher SC': 'Karlsruhe',
'FC Hansa Rostock': 'Hansa Rostock',
'MSV Duisburg': 'Duisburg',
'Alemannia Aachen': 'Aachen'
 }
market_values_bl.club = market_values_bl.club.replace(map_team_names_dict)
market_value_column_name = f"Total market value"
# market_value_column_name = f"Value Sep 15, 20{season}"
# market_value_column_name = f"Value Jul 10, 20{season}"
market_values_bl = market_values_bl.dropna(subset=['club'])
market_values_bl[market_value_column_name] = market_values_bl[market_value_column_name].apply(lambda x: float(re.search(r"\d+\.\d{2}", x).group()))
market_values_bl

,club,Squad,ø age,Foreigners,ø market value,Total market value
0,Bayern Munich,30,26. Mai,16,€6.02m,180.73
1,Werder Bremen,33,24. Sep,14,€4.77m,157.25
2,Schalke 04,30,24. Jul,14,€3.55m,106.60
3,Hamburg,36,24. Sep,19,€2.83m,101.83
4,Stuttgart,32,25. Mrz,11,€3.03m,97.08
5,Dortmund,32,24.0,14,€2.49m,79.78
6,Leverkusen,28,25. Jan,14,€2.65m,74.08
7,Hertha,31,25. Apr,18,€2.14m,66.40
8,Nurnberg,31,25. Mai,20,€2.11m,65.45
9,Wolfsburg,30,26. Sep,21,€1.86m,55.90


In [207]:
# add market values to df
# df_bundesliga_with_form['marketValueHT'] = market_values_bl[market_values_bl.club==df_bundesliga_with_form.HomeTeam]

In [208]:
market_values_bl[market_values_bl.club=='Bayern Munich']
rows.HomeTeam
rows.AwayTeam


'Werder Bremen'

In [209]:
# add market values to df
market_value_HT = []
market_value_AT = []
for index, rows in df_bundesliga_with_form.iterrows():
    mw_ht = market_values_bl[market_values_bl.club==rows.HomeTeam]
    market_value_HT.append(float(mw_ht[market_value_column_name]))
    mw_at = market_values_bl[market_values_bl.club==rows.AwayTeam]
    market_value_AT.append(float(mw_at[market_value_column_name]))

df_bundesliga_with_form["MarketValueHT"] = market_value_HT
df_bundesliga_with_form["MarketValueAT"] = market_value_AT
df_bundesliga_with_form["MarketValueDiff"] = df_bundesliga_with_form.loc[:, "MarketValueHT"] - df_bundesliga_with_form.loc[:, "MarketValueAT"]
df_bundesliga_with_form["MarketValueQuot"] = df_bundesliga_with_form.loc[:, "MarketValueHT"]/df_bundesliga_with_form.loc[:, "MarketValueAT"]

In [210]:
# save modified bl game information of one season in new file
df_bundesliga_with_form.to_csv(f'Data_BL_modified//bundesliga_data_modified_season_{season}.csv')

Neue Features mit dazunehmen
* wie Marktwert (am Anfang der Saison, beziehen von z.B. Transfermarkt; Alternativ MW der Startaufstellung, aber schwerer zu bekommen; sowohl absoluter MW als auch Durchschnitt kann verwendet werden)
* Direkter Vergleich der beiden Teams (vgl. Artikel Poisson Verteilung)
* Form der letzten Spiele? (kurzfristige Form im Vergleich zur ELO als langfristige Form)
* Tabellenplatz

# Informations about last Season
## primary qustion: How often wins a team?

In [211]:
# winning_team home/away
winning_team = bundesliga_21_22_df['FTR'].value_counts()
print(f"Gewinnchancen \nHome Team: {winning_team['H']/winning_team.sum()}")
print(f"Away Team: {winning_team['A']/winning_team.sum()}")
print(f"Draw: {winning_team['D']/winning_team.sum()}")

Gewinnchancen 
Home Team: 0.43790849673202614
Away Team: 0.30392156862745096
Draw: 0.2581699346405229


In [212]:
# winning_team according to last matches
print(f"Gewinnchancen:")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["PDiff3Matches"]>0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["PDiff3Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 3 Spielen: {len(winning_team)/len(df_bundesliga_with_form['PDiff3Matches'].dropna())}")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["PDiff5Matches"]>0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["PDiff5Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 5 Spielen: {len(winning_team)/len(df_bundesliga_with_form['PDiff5Matches'].dropna())}")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["PDiff10Matches"]>0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["PDiff10Matches"]<0)]
print(f"Team mit der besseren Form in den vergangenen 10 Spielen: {len(winning_team)/len(df_bundesliga_with_form['PDiff10Matches'].dropna())}")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["PDiffAllMatches"]>0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["PDiffAllMatches"]<0)]
print(f"Team mit der besseren Tabellenplatzierung: {len(winning_team)/len(df_bundesliga_with_form['PDiffAllMatches'].dropna())}")

Gewinnchancen:
Team mit der besseren Form in den vergangenen 3 Spielen: 0.36200716845878134
Team mit der besseren Form in den vergangenen 5 Spielen: 0.41379310344827586
Team mit der besseren Form in den vergangenen 10 Spielen: 0.4212962962962963
Team mit der besseren Tabellenplatzierung: 0.38235294117647056


In [213]:
# winning_team according to market value
print(f"Gewinnchancen:")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["MarketValueDiff"]>0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["MarketValueDiff"]<0)]
print(f"Team mit dem höheren Marktwert: {len(winning_team)/len(df_bundesliga_with_form['MarketValueDiff'].dropna())}")
print(f"Gewinnchancen Underdog:")
winning_team = df_bundesliga_with_form[(df_bundesliga_with_form['FTR']=='H')&(df_bundesliga_with_form["MarketValueDiff"]<0)|(df_bundesliga_with_form['FTR']=='A')&(df_bundesliga_with_form["MarketValueDiff"]>0)]
print(f"Team mit dem niedrigeren Marktwert: {len(winning_team)/len(df_bundesliga_with_form['MarketValueDiff'].dropna())}")

Gewinnchancen:
Team mit dem höheren Marktwert: 0.4444444444444444
Gewinnchancen Underdog:
Team mit dem niedrigeren Marktwert: 0.2908496732026144
